# ２章 ＥＸＣＥＬデータの加工・可視化を行う２０本ノック

## ノック２１：Excelデータを読み込んでみよう

In [ ]:
import pandas as pd
data = pd.read_excel('data/1-2-2020.xlsx')
data.head()

In [ ]:
data.tail()

In [ ]:
data = pd.read_excel('data/1-2-2020.xlsx', skiprows=4, header=None)
data.head()

In [ ]:
data.drop(data.tail(4).index, inplace=True)
data.tail()

## ノック２２：カラムを抽出して付与してみよう

In [ ]:
col_data = pd.read_excel('data/1-2-2020.xlsx', skiprows=1, header=None)
col_data = col_data.head(3)
col_data

In [ ]:
col_data.iloc[1,1:].fillna(col_data.iloc[0,1:], inplace=True)
col_data.iloc[1, 1:] = col_data.iloc[1, 1:].str.replace('発電所','')
col_data

In [ ]:
for i in col_data.columns:
  if i < col_data.columns.max():
    col_data[i + 1].fillna(col_data[i], inplace=True)
col_data

In [ ]:
col_data.replace('〔バイオマス〕','バイオマス', inplace=True)
col_data.replace('〔廃棄物〕','廃棄物', inplace=True)
col_data

In [ ]:
tg_col = '_'.join(list(col_data[0].dropna()))
print(tg_col)
tg_col = '_'.join(list(col_data[1].dropna()))
print(tg_col)

In [ ]:
cols = []
for i in col_data.columns:
  tg_col = '_'.join(list(col_data[i].dropna()))
  cols.append(tg_col)
cols

In [ ]:
data.columns = cols
data.head()

## ノック２３：全シートのデータを読み込んでみよう

In [ ]:
xl = pd.ExcelFile('data/1-2-2020.xlsx')
sheets = xl.sheet_names
sheets

In [ ]:
 data = xl.parse(sheets[0],  skiprows=4, header=None)
 data.drop(data.tail(4).index, inplace=True)
 data.columns = cols
 data.head()

In [ ]:
datas = []
for sheet in sheets:
  data = xl.parse(sheet,  skiprows=4, header=None)
  data.drop(data.tail(4).index, inplace=True)
  data.columns = cols
  data['年月'] = sheet
  datas.append(data)
datas

In [ ]:
datas = pd.concat(datas, ignore_index=True)
datas.head()

## ノック２４：データの値を計算で修正しよう

In [ ]:
datas['火力発電所_火力_発電所数'] = datas['火力発電所_火力_発電所数'] - datas['新エネルギー等発電所_バイオマス_発電所数'] - datas['新エネルギー等発電所_廃棄物_発電所数']
datas['火力発電所_火力_最大出力計'] = datas['火力発電所_火力_最大出力計'] - datas['新エネルギー等発電所_バイオマス_最大出力計'] - datas['新エネルギー等発電所_廃棄物_最大出力計']
datas.head()

## ノック２５：必要なカラムだけに絞り込もう

In [ ]:
datas.drop(['合計_合計_発電所数', '合計_合計_最大出力計', '新エネルギー等発電所_計_発電所数', '新エネルギー等発電所_計_最大出力計'], axis=1, inplace=True)
datas.head()

## ノック２６：縦持ちデータを作成しよう

In [ ]:
datas_v = pd.melt(datas, id_vars=['都道府県','年月'], var_name="変数名",value_name="値")
datas_v.head()

## ノック２７：縦持ちデータを整形しよう

In [ ]:
var_data = datas_v['変数名'].str.split('_', expand=True)
var_data.head()

In [ ]:
var_data.columns = ['発電所種別', '発電種別', '項目']
datas_v = pd.concat([datas_v, var_data], axis=1)
datas_v.drop(['変数名'], axis=1, inplace=True)
datas_v.head()

## ノック２８：発電実績データを加工しよう

In [ ]:
capacity_data = pd.read_excel('data/2-2-2020.xlsx')
display(capacity_data.head())
display(capacity_data.tail())

In [ ]:
col_ca_data = pd.read_excel('data/2-2-2020.xlsx', skiprows=1, header=None)
col_ca_data = col_ca_data.head(3)

col_ca_data.iloc[1,1:].fillna(col_ca_data.iloc[0,1:], inplace=True)
col_ca_data.iloc[1, 1:] = col_ca_data.iloc[1, 1:].str.replace('発電所','')

for i in col_ca_data.columns:
  if i < col_ca_data.columns.max():
    col_ca_data[i + 1].fillna(col_ca_data[i], inplace=True)
col_ca_data.replace('〔バイオマス〕','バイオマス', inplace=True)
col_ca_data.replace('〔廃棄物〕','廃棄物', inplace=True)

cols_ca = []
for i in col_ca_data.columns:
  tg_col = '_'.join(list(col_ca_data[i].dropna()))
  cols_ca.append(tg_col)
cols_ca

In [ ]:
xl_ca = pd.ExcelFile('data/2-2-2020.xlsx')
sheets = xl_ca.sheet_names
ca_datas = []
for sheet in sheets:
  capacity_data = xl_ca.parse(sheet,  skiprows=4, header=None)
  capacity_data = capacity_data.head(47)
  capacity_data.columns = cols_ca
  capacity_data['年月'] = sheet
  ca_datas.append(capacity_data)
ca_datas = pd.concat(ca_datas, ignore_index=True)
ca_datas.head()

In [ ]:
ca_datas['火力発電所_火力_電力量'] = ca_datas['火力発電所_火力_電力量'] - ca_datas['新エネルギー等発電所_バイオマス_電力量'] - ca_datas['新エネルギー等発電所_廃棄物_電力量']
ca_datas.drop(['合計_合計_電力量','新エネルギー等発電所_計_電力量'], axis=1, inplace=True)
ca_datas_v = pd.melt(ca_datas, id_vars=['都道府県','年月'], var_name="変数名",value_name="値")
var_data = ca_datas_v['変数名'].str.split('_', expand=True)
var_data.columns = ['発電所種別', '発電種別', '項目']
ca_datas_v = pd.concat([ca_datas_v, var_data], axis=1)
ca_datas_v.drop(['変数名'], axis=1, inplace=True)
ca_datas_v.head()

## ノック２９：可視化用データを仕上げよう

In [ ]:
datas_v_all = pd.concat([datas_v, ca_datas_v], ignore_index=True)
display(datas_v_all.head())
display(datas_v_all.tail())

In [ ]:
pd.pivot_table(datas_v_all.loc[datas_v_all['年月']=='2020.4'], index='発電所種別', columns='項目', values='値', aggfunc='sum')

## ノック３０：データ分布をヒストグラムで可視化してみよう



In [ ]:
%%shell
pip install japanize-matplotlib

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

plt.figure(figsize=(20, 10))
sns.histplot(datas_v_all.loc[datas_v_all['項目']=='発電所数'])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
viz_data = datas_v_all.loc[datas_v_all['値']!=0]
sns.histplot(viz_data.loc[viz_data['項目']=='発電所数'], ax=axes[0])
sns.histplot(viz_data.loc[viz_data['項目']=='最大出力計'], ax=axes[1])
sns.histplot(viz_data.loc[viz_data['項目']=='電力量'], ax=axes[2])

## ノック３１：データ分布を箱ひげ図で可視化してみよう

In [ ]:
plt.figure(figsize=(10, 10))
viz_data = datas_v_all.loc[(datas_v_all['項目']=='発電所数')&(datas_v_all['値']!=0)]
sns.boxplot(y=viz_data['値'])

In [ ]:
plt.figure(figsize=(30, 10))
sns.boxplot(x=viz_data['発電種別'], y=viz_data['値'])

## ノック３２：最近の発電量を可視化してみよう

In [ ]:
viz_data = datas_v_all[['発電種別','値']].loc[(datas_v_all['項目']=='電力量')&(datas_v_all['年月']=='2021.1')]
viz_data = viz_data.groupby('発電種別', as_index=False).sum()
viz_data

In [ ]:
sns.barplot(x=viz_data['発電種別'], y=viz_data['値'])

## ノック３３：先月の発電量とあわせて可視化してみよう

In [ ]:
viz_data = datas_v_all[['発電種別','年月','値']].loc[(datas_v_all['項目']=='電力量')]
viz_data = viz_data.groupby(['発電種別','年月'],as_index=False).sum()
viz_data.head()

In [ ]:
viz_data = viz_data.loc[(viz_data['年月']=='2020.12')|(viz_data['年月']=='2021.1')]
sns.barplot(x=viz_data['発電種別'], y=viz_data['値'], hue=viz_data['年月'])

## ノック３４：電力の時系列変化を可視化してみよう

In [ ]:
plt.figure(figsize=(15, 5))
viz_data = datas_v_all[['発電種別','年月','値']].loc[(datas_v_all['項目']=='電力量')]
viz_data = viz_data.groupby('年月',as_index=False).sum()
viz_data['年月'] = pd.to_datetime(viz_data['年月'])
sns.lineplot(x=viz_data['年月'], y=viz_data["値"])

In [ ]:
plt.figure(figsize=(15, 5))
viz_data = datas_v_all[['発電種別','年月','値']].loc[(datas_v_all['項目']=='電力量')]
viz_data = viz_data.groupby(['発電種別','年月'],as_index=False).sum()
viz_data['年月'] = pd.to_datetime(viz_data['年月'])
sns.lineplot(x=viz_data['年月'], y=viz_data["値"], hue=viz_data['発電種別'])

## ノック３５：電力の割合を可視化してみよう

In [ ]:
viz_data = datas_v_all.loc[(datas_v_all['項目']=='電力量')&(datas_v_all['年月']=='2021.1')]
viz_data = viz_data[['発電種別','値']].groupby('発電種別').sum()
viz_data['割合'] = viz_data['値'] / viz_data['値'].sum()
viz_data

In [ ]:
viz_data.T.loc[['割合']].plot(kind='bar', stacked=True)

## ノック３６：電力量の多い都道府県を比較してみよう

In [ ]:
viz_data = datas_v_all.loc[datas_v_all['項目']=='電力量']
viz_data = viz_data[['都道府県','値']].groupby('都道府県', as_index=False).sum()
viz_data.sort_values('値', inplace=True, ascending=False)
viz_data.head(5)

In [ ]:
plt.figure(figsize=(15, 5))
viz_data = datas_v_all[['都道府県','年月','値']].loc[(datas_v_all['項目']=='電力量')&((datas_v_all['都道府県']=='神奈川県')|(datas_v_all['都道府県']=='千葉県'))]
viz_data = viz_data.groupby(['年月', '都道府県'],as_index=False).sum()
viz_data['年月'] = pd.to_datetime(viz_data['年月'])
sns.lineplot(x=viz_data['年月'], y=viz_data["値"], hue=viz_data['都道府県'])

In [ ]:
viz_data_num = datas_v_all[['都道府県','年月','値']].loc[(datas_v_all['項目']=='発電所数')&((datas_v_all['都道府県']=='神奈川県')|(datas_v_all['都道府県']=='千葉県'))]
viz_data_num = viz_data_num.groupby(['年月', '都道府県'],as_index=False).sum()
viz_data_num['年月'] = pd.to_datetime(viz_data_num['年月'])
viz_data.rename(columns={'値':'電力量'}, inplace=True)
viz_data_num.rename(columns={'値':'発電所数'}, inplace=True)
viz_data_join = pd.merge(viz_data, viz_data_num, on=['年月', '都道府県'], how='left')
viz_data_join.head()

In [ ]:
sns.relplot(x=viz_data_join['年月'],  y=viz_data_join['電力量'],
            hue=viz_data_join['都道府県'], size=viz_data_join['発電所数'],
            alpha=0.5, height=5, aspect=2)

## ノック３７：都道府県、年月別の電力量を可視化してみよう

In [ ]:
viz_data = datas_v_all[['都道府県','年月','値']].loc[datas_v_all['項目']=='電力量']
viz_data = viz_data.groupby(['年月', '都道府県'],as_index=False).sum()
viz_data['年月'] = pd.to_datetime(viz_data['年月']).dt.date

viz_data = viz_data.pivot_table(values='値', columns='年月', index='都道府県')
viz_data.head(5)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(viz_data)

## ノック３８：変数の関係性を可視化してみよう

In [ ]:
viz_data = datas.drop(['都道府県','年月'],axis=1)
viz_data.head(5)

In [ ]:
sns.scatterplot(x=viz_data['水力発電所_水力_発電所数'], y=viz_data['水力発電所_水力_最大出力計'])

In [ ]:
sns.jointplot(x=viz_data['水力発電所_水力_発電所数'], y=viz_data['水力発電所_水力_最大出力計'])

In [ ]:
sns.pairplot(viz_data.iloc[:,0:4])

## ノック３９：データを整形してExcel形式で出力しよう

In [ ]:
output = datas_v_all.pivot_table(values='値', columns='項目', index=['年月','都道府県'], aggfunc='sum')
output.head()

In [ ]:
output.to_excel('data/summary_data.xlsx')

## ノック４０：シート別にExcelデータを出力しよう

In [ ]:
target = '北海道'
tmp = datas_v_all.loc[datas_v_all['都道府県']==target]
tmp = tmp.pivot_table(values='値', columns=['発電種別','項目'], index=['年月'], aggfunc='sum')
tmp.head(5)

In [ ]:
writer = pd.ExcelWriter('data/detail_data.xlsx', mode='w')

for target in datas_v_all['都道府県'].unique():
  tmp = datas_v_all.loc[datas_v_all['都道府県']==target]
  tmp = tmp.pivot_table(values='値', columns=['発電種別','項目'], index=['年月'], aggfunc='sum')
  with writer as w:
    tmp.to_excel(w, sheet_name=target)